# WNTR Basic Tutorial
The following tutorial illustrates basic use of WNTR, including use of the `WaterNetworkModel` object, the ability to read/write model files to other formats, run hydraulic and water quality simulations, skeletonize water network models.

## Imports
Install and import WNTR and additional Python packages that are needed for the tutorial
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Matplotlib is required to create graphics

In [ ]:
#!pip install wntr numpy scipy networkx geopandas matplotlib 

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import wntr

Create directories to hold results

## Units
WNTR uses **SI (International System) units (length in meters, time in seconds, mass in kilograms)**.  See https://usepa.github.io/WNTR/units.html for more details.

# Water Network Model

The `WaterNetworkModel` object defines the water distribution system and simulation options. The object can be created from an EPANET INP file.

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('networks/Net3.inp')

In [ ]:
# Print a basic description of the model.  The level can be 0, 1, or 2 and defines the level of detail included in the description.
wn.describe(level=1)

In [ ]:
# List properties and methods associated with the WaterNetworkModel (omitting private underscore names)
[name for name in dir(wn) if not name.startswith('_')]

In [ ]:
# Plot a basic network graphic
ax = wntr.graphics.plot_network(wn)

## Nodes
Nodes define junctions, tanks, and reservoirs

In [ ]:
# Print the names of all junctions, tanks, and reservoirs
print("Node names", wn.node_name_list)

In [ ]:
# Print the names of just tanks
print("Tank names", wn.tank_name_list)

In [ ]:
# Get a tank object
tank = wn.get_node('1')
print(type(tank))
tank

In [ ]:
# List properties and methods associated with the tank (omitting private underscore names)
[name for name in dir(tank) if not name.startswith('_')]

In [ ]:
# print tank 1 initial level




In [ ]:
# Change the max level of a tank
print("Original max level", tank.max_level)
tank.max_level = 10
print("New max level", tank.max_level)

In [ ]:
# Add a junction to the WaterNetworkModel
wn.add_junction('new_junction', base_demand=0.0, demand_pattern=None, elevation=0.0, coordinates=None, demand_category=None)
print(wn.junction_name_list)

In [ ]:
# Remove a junction from the WaterNetworkModel
wn.remove_node('new_junction')
print(wn.junction_name_list)

## Links
Links define pipes, pumps, and valves

In [ ]:
# Print the names of all pipes, pumps, and valves
print("Link names", wn.link_name_list)

In [ ]:
# Print the names of just head pumps
print("Head pump names", wn.head_pump_name_list)

In [ ]:
# Get the name of links connected to a specific node
connected_links = wn.get_links_for_node('229')
print('Links connected to node 229 =', connected_links)

In [ ]:
# Get a pipe object
pipe = wn.get_link('105')
print(type(pipe))
pipe

In [ ]:
# List properties and methods associated with the pipe (omitting private underscore names)
[name for name in dir(pipe) if not name.startswith('_')]

In [ ]:
# get diameter of pipe 60



In [ ]:
# Change the diameter of a pipe
print("Original diameter", pipe.diameter)
pipe.diameter = 10
print("New diameter", pipe.diameter)

In [ ]:
# Add a pipe to the WaterNetworkModel
wn.add_pipe(name="new_pipe", start_node_name="10", end_node_name="123", length=304.8, diameter=0.3048, roughness=100, minor_loss=0.0, initial_status='OPEN', check_valve=False)
print(wn.pipe_name_list)

In [ ]:
pipe = wn.get_link('new_pipe')
print(type(pipe))
print(pipe)
print(pipe.initial_status)


In [ ]:
# Remove a pipe from the WaterNetworkModel
wn.remove_link("new_pipe")
print(wn.pipe_name_list)

## Demands and Patterns
Junctions can have multiple demands which are stored as Timeseries objects in a `demand_timeseries_list`. Each Timeseries contains a base value, pattern, and category.  Patterns contain multipliers and the pattern timestep.  

The following example illustrates how to
* Get demand and patterns of a junction
* Querry for demands at junctions
* Add demands to a junction
* Modify demand base value and pattern
* Remove demands from a junction
* Plot expected and simulated demands

In [ ]:
# Get the demands on Junction 15
junction = wn.get_node('15')
junction.demand_timeseries_list

In [ ]:
# Get the demands on Junction 101
junction = wn.get_node('101')
junction.demand_timeseries_list

In [ ]:
# Get the pattern associated with the demand
pattern = wn.get_pattern(junction.demand_timeseries_list[0].pattern_name)
pattern

In [ ]:
# Modify the base value of the demand
junction.demand_timeseries_list[0].base_value = 0.005

# Add a new pattern to the model
wn.add_pattern('New', [1,1,1,0,0,0,1,0,0.5,0.5,0.5,1])

# Use the new pattern to modify the junction demand
junction.demand_timeseries_list[0].pattern_name = "New"
print(junction.demand_timeseries_list)

In [ ]:
# check the new pattern assigned to the junction
print(junction.name)
pattern = wn.get_pattern(junction.demand_timeseries_list[0].pattern_name)
pattern

## Curves
Curves define pump head curves, tank volume curves, and pump efficiency curves.  The following example illustrates how to work with pump head curves and tank volume curves.

In [ ]:
# Get a head pump object and plot the head pump curve
pump = wn.get_link('10')
print(type(pump))
ax = wntr.graphics.plot_pump_curve(pump)

In [ ]:
# Get the head curve and print the points
pump_curve_name = pump.pump_curve_name
curve = wn.get_curve(pump_curve_name)
curve.points

In [ ]:
# Modify the curve points and re-plot the pump curve
curve.points = [(0.10, 20)]
ax = wntr.graphics.plot_pump_curve(pump)

In [ ]:
# Add a tank volume curve to the model and assign it to a tank
wn.add_curve('new_tank_curve', 'VOLUME', [
   (1,  0),
   (2,  60),
   (3,  188),
   (4,  372),
   (5,  596),
   (6,  848),
   (7,  1114),
   (8,  1379),
   (9,  1631),
   (10, 1856),
   (11, 2039),
   (12, 2168),
   (13, 2228)])
tank = wn.get_node('2')
tank.vol_curve_name = 'new_tank_curve'
ax = wntr.graphics.plot_tank_volume_curve(tank)

## Controls

Controls define conditions and actions that operate pipes, pumps, and valves.  WNTR includes support for EPANET controls and rules (note that both are stored as WNTR controls). As with EPANET, controls are evaluated after each simulation timestep, while rules are evaluated after each rule timestep (see `wn.options.time`). The method `convert_controls_to_rules` can be used to convert controls to rules, which can help avoid unintended behavior when controls and rules are both used in complex simulations.

In [ ]:
# Get a list of control names
wn.control_name_list

In [ ]:
# Print all controls
for name, controls in wn.controls():
    print(name, controls)

In [ ]:
# Get a specific control object
control = wn.get_control('control 18')
print(control)

## Queries
Queries return attributes of nodes and links.  Comparison operations (like >, =) can be used to return a subset of attributes that meet specific criteria.

In [ ]:
# Return all pipe diameters (no comparison operator used in the query) 
all_pipe_diameters = wn.query_link_attribute('diameter')
all_pipe_diameters.head()

In [ ]:
# Return pipes diameters > 12 inches
large_pipe_diameters = wn.query_link_attribute('diameter', np.greater, 12*0.0254)
print("Number of pipes:", len(all_pipe_diameters))
print("Number of pipes > 12 inches:", len(large_pipe_diameters))

In [ ]:
# Plot pipes diameters > 12 inches
ax = wntr.graphics.plot_network(wn, link_attribute=large_pipe_diameters, node_size=0, link_width=2, title="Pipes with diameter > 12 inches")

## Loops and generators
Loops and generators are commonly used to modify network components or run stochastic simulations

In [ ]:
# Loop over tank names and objects with a generator
for name, tank in wn.tanks():
    print("Max level for tank", name, "=", tank.max_level)

# Hydraulic Simulations

WNTR includes two simulators: the `EpanetSimulator` and the `WNTRSimulator`.  Both include the ability to run pressure dependent demand (PDD) or demand-driven (DD) hydraulic simulation.  Only the EpanetSimulator runs water quality simulations.

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('networks/Net3.inp')

## Simulation options
Simulation options include options related to simulation time, hydraulics, water quality, reactions, energy calculations, reporting, graphics, and user/custom options.

In [ ]:
# Print the WaterNetworkModel options
wn.options

In [ ]:
# Print the time options
wn.options.time

In [ ]:
# Print the simulation duration
wn.options.time.duration/3600/24

In [ ]:
# Change the simulation duration to 4 days
wn.options.time.duration = 4*24*3600 # seconds
print(wn.options.time)

In [ ]:
# Print the simulation duration
wn.options.time.duration/3600/24

## EPANET and WNTR Simulators

In [ ]:
# Simulate hydraulics using EPANET
sim = wntr.sim.EpanetSimulator(wn)
results_EPANET = sim.run_sim()

In [ ]:
# Simulate hydraulics using the WNTRSimulator
sim = wntr.sim.WNTRSimulator(wn)
results_WNTR = sim.run_sim()

## Simulation results
Simulation results are stored in an object which includes a dictionary of DataFrames for nodes and a dictionary of DataFrames for links.  Each DataFrame is indexed by time (in seconds) and the columns are node or link names.

In [ ]:
# Print available node results
results_EPANET.node.keys()

In [ ]:
# Print available link results
results_EPANET.link.keys()

In [ ]:
# View EpanetSimulator pressure results
results_EPANET.node['pressure'].head()

In [ ]:
# Compare EpanetSimulator and WNTRSimulator pressure results
diff = results_EPANET.node['pressure'] - results_WNTR.node['pressure']
ax = diff.max(axis=1).plot(title='Max difference in pressure')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Pressure difference (m)')

In [ ]:
# Plot timeseries of tank levels
tank_levels = results_EPANET.node['pressure'].loc[:,wn.tank_name_list]
ax = tank_levels.plot(title='Tank level')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Tank Level (m)')

In [ ]:
# Plot timeseries of pump flowrates
pump_flowrates = results_EPANET.link['flowrate'].loc[:,wn.pump_name_list]
ax = pump_flowrates.plot(title='Pump flowrate')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Pump flowrate (m$^3$/s)')

In [ ]:
# Plot pressure at hour 5 on the network
pressure_at_5hr = results_EPANET.node['pressure'].loc[5*3600, :]
ax = wntr.graphics.plot_network(wn, node_attribute=pressure_at_5hr, node_size=30, title='Pressure at 5 hours')